In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from generate_ddf_sched import generate_ddf
from lsst.sims.cloudModel import CloudData
from astropy.time import Time


In [2]:
# Load up the cloud model and set the cloud limit
cloud_limit = 0.3
mjd_start=59853.5
mjd_start_time = Time(mjd_start, format='mjd')    
cloud_data = CloudData(mjd_start_time, offset_year=0)

In [3]:
# generate a list of times we want to observe some sequence
mjd_observe, ra ,dec, observe_sequence, m5 = generate_ddf('DD:COSMOS', nyears=5.5)

/Users/yoachim/git_repos/sims_seeingModel/python/lsst/sims/seeingModel/seeingModel.py:126: RuntimeWarning: invalid value encountered in power
  airmass_correction = np.power(airmass, 0.6)
/Users/yoachim/git_repos/sims_featureScheduler_runs_technical/ddf_ahead/generate_ddf_sched.py:113: RuntimeWarning: invalid value encountered in greater
  (ddf_m5 > g_m5_limit))


In [4]:
# I only grabbed the 5-sigma depth in g, so let's just make the list of observations in g
observationStartMJD = []
m5s = []
mjds = Time(mjd_observe, format='mjd')

clouded_out = 0
for i,mjd in enumerate(mjds):
    if cloud_data(mjd[0]) < cloud_limit:
        for obs in observe_sequence:
            if obs['filter'] == 'g':
                observationStartMJD.append(mjd[0].value)
                m5s.append(m5[i])
    else:
        clouded_out += 1

In [5]:
clouded_out

11

In [6]:
# Wrap into MAF numpy array with the expected dtype names
names = ['observationStartMJD', 'fieldRA', 'fieldDec', 'filter',
         'visitExposureTime', 'fiveSigmaDepth',
        'seeingFwhmGeom']
types = [float, float, float, '|U1', float, float, float]
dataSlice = np.empty(len(observationStartMJD), dtype=list(zip(names,types)))

dataSlice['observationStartMJD'] = observationStartMJD
dataSlice['fieldRA'] = np.degrees(ra.value)
dataSlice['fieldDec'] = np.degrees(dec.value)
dataSlice['filter'] = 'g'
dataSlice['visitExposureTime'] = 30.
dataSlice['fiveSigmaDepth'] = m5s
dataSlice['seeingFwhmGeom'] = 1.2  # Just plug in a number, could have saved from before maybe. Or if I'd saved the airmass I could look it up

In [7]:
from lsst.sims.maf.metrics import Coaddm5Metric, ProperMotionMetric, ExgalM5
from lsst.sims.maf.slicers import UserPointsSlicer
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db

outDir='temp'
resultsDb = db.ResultsDb(outDir=outDir)

In [13]:
# For things where we don't need slicePoint information, we can just run the metric itself
metric = Coaddm5Metric()
print('coadded depth', metric.run(dataSlice))
metric = ProperMotionMetric()
print('proper motion uncertainty', metric.run(dataSlice))

coadded depth 28.20813108592443
proper motion uncertainty 0.18281910906209978


In [9]:
# OK, if we wanted to use all of the MAF framework
bundleList = []
slicer = UserPointsSlicer(ra=np.degrees(ra.value), dec=np.degrees(dec.value))
metric = ExgalM5(lsstFilter='g')
# Note, the sql constraint doesn't do anything since we're going to skip the DB querry, so up to the user to make sure 
# the data passed in matches the sql constraint
sql = 'filter = "g"'  
# Here's how we could impose the filter selection
ds_indx = np.where(dataSlice['filter'] == 'g')[0]
bundleList.append(metricBundles.MetricBundle(metric,slicer,sql))

metric = Coaddm5Metric()
bundleList.append(metricBundles.MetricBundle(metric,slicer,sql))


In [10]:
bg = metricBundles.MetricBundleGroup(bundleList, None, outDir=outDir, resultsDb=resultsDb)

/Users/yoachim/git_repos/sims_maf/python/lsst/sims/maf/metricBundles/metricBundleGroup.py:110: UserWarning: Warning: dbObj should be an instantiated Database (or child) object.
  warnings.warn('Warning: dbObj should be an instantiated Database (or child) object.')


In [11]:
# Ugh, need to update MAF a little to skip this
bg.setCurrent(bg.constraints[0])
bg.runCurrent(bg.constraints[0], simData=dataSlice[ds_indx])
# Should update so this can run as: bg.runAll(simData=dataSlice)

Running:  ['opsim_ExgalM5_fiveSigmaDepth_g_USER']
Completed metric generation.
Running:  ['opsim_CoaddM5_g_USER']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [12]:
bundleList[0].metricValues, bundleList[1].metricValues

(masked_array(data=[28.131404951335057],
              mask=[False],
        fill_value=-666.0), masked_array(data=[28.20813108592443],
              mask=[False],
        fill_value=-666.0))